Report here ...

In [ ]:
import pandas as pd

ds=pd.read_csv('../input/airplane-crash-data-since-1908/Airplane_Crashes_and_Fatalities_Since_1908_20190820105639.csv')
print("The shape of the dataset: ", ds.shape)
ds.head()

In [ ]:
import seaborn as sn
import matplotlib.pyplot as plt

corrMatrix=ds.corr()
sn.heatmap(corrMatrix, annot=True)
plt.show()

In [ ]:
print("Number of rows before removing the missing data:")
print(ds.shape[0])
#dsDrop = ds.dropna()

ds = ds.dropna() #________added

print("Number of rows after removing the missing data:")
print(ds.shape[0]) #________added
#print(dsDrop.shape[0])

In [ ]:
dso = ds.drop(['Date', 'Time', 'Location', 'Operator', 'Flight #', 'Route', 'AC Type', 'Registration', 'cn/ln', 'Summary'],axis=1)
print(dso.shape)
QL1=dso.quantile(0.25)
QL3=dso.quantile(0.75)
IQRL=QL3-QL1
FLL=QL1-(1.5*IQRL)
FHL=QL3+(1.5*IQRL)
dso=dso.loc[(dso>FLL).all(axis=1) & (dso<FHL).all(axis=1)]
print("Number of rows after removing the outliers:")
print(dso.shape[0])

In [ ]:
dup = ds.duplicated()
print('Number of duplicate rows = %d' % (dup.sum()))
ds=ds.drop_duplicates()
print("Number of rows after removing the duplicate data:")
print(ds.shape[0])

In [ ]:
ds.dtypes

In [ ]:
#Replace NULL with NaN
import numpy as np

#drop_Sum = ds #save Summary column

#ds = ds.drop(['Summary'],axis=1)

#print('Number of missing values:')
#for col in ds.columns:
#    print(col +': ' + str(ds[col].isna().sum()))
    

In [ ]:
#ds

In [ ]:
#ds.replace('NULL',np.NaN)
#ds

In [ ]:
ds.dtypes

In [ ]:
#Fill empty rows with NULL 
#ds["Summary"].fillna("NULL", inplace = True) 
#ds["Summary"]

In [ ]:
#Noise Removal

import string 

i = 0
punct = "\n\r"+string.punctuation

ds["Summary"] = ds["Summary"].str.translate(str.maketrans('','',punct))  
ds

In [ ]:
#Stop Words Removal
from nltk.corpus import stopwords

stop = stopwords.words('english')
ds["Summary"] = ds["Summary"].str.lower().str.split()
ds["Summary"]= ds["Summary"].apply(lambda x: ' '.join([item for item in x if item not in stop]))

In [ ]:
#TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

X= ds.Summary.fillna(' ')

vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(ds["Summary"])
feature_names = vectorizer.get_feature_names()
print (feature_names)
print ("Number of words = ",len(feature_names))

In [ ]:
vectorizer = TfidfVectorizer(stop_words='english', token_pattern=r'(?u)\b[A-Za-z]+\b')
vectors = vectorizer.fit_transform(ds["Summary"])

feature_names = vectorizer.get_feature_names()

print(feature_names)
print("number of words = ", len(feature_names))

In [ ]:
#Stemming

import re
import nltk
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()
def tokenize(str_input):
    
    words = re.sub(r"(?u)[^A-Za-z]", " ", str_input).lower().split()
    words = [stemmer.stem(word) for word in words]
    return words


vectorizer = TfidfVectorizer(tokenizer=tokenize, stop_words='english')
vectors = vectorizer.fit_transform(ds["Summary"])
feature_names = vectorizer.get_feature_names()

print(feature_names)
print("number of words = ", len(feature_names))

In [ ]:
#lemmatization
from nltk.stem import WordNetLemmatizer

wordnet_lemmatizer = WordNetLemmatizer()

def tokenize(str_input):
    
    words = re.sub(r"(?u)[^A-Za-z]", " ", str_input).lower().split(" ")
    words = [stemmer.stem(word) for word in words if len(word)>2]
    words = [wordnet_lemmatizer.lemmatize(word) for word in words if len(word)>2]
    return words

vectorizer = TfidfVectorizer(tokenizer=tokenize, stop_words='english')
vectors = vectorizer.fit_transform(ds["Summary"])
feature_names = vectorizer.get_feature_names()

print(feature_names)
print("number of words = ", len(feature_names))

In [ ]:
#Convert categorical data to numerical data
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
ds['Date'] = le.fit_transform (ds['Date'])
ds['Time'] = le.fit_transform (ds['Time'])
ds['Location'] = le.fit_transform (ds['Location'])
ds['Operator'] = le.fit_transform (ds['Operator'])
ds['Flight #'] = le.fit_transform (ds['Flight #'])
ds['Route'] = le.fit_transform (ds['Route'])
ds['AC Type'] = le.fit_transform (ds['AC Type'])
ds['Registration'] = le.fit_transform (ds['Registration'])
ds['cn/ln'] = le.fit_transform (ds['cn/ln'])
ds['Summary'] = le.fit_transform (ds['Summary'])

In [ ]:
#Sum-of-Squared Errors (SSE)

from sklearn import cluster
import matplotlib.pyplot as plt
%matplotlib inline

numClusters = [1,2,3,4,5,6]
SSE = []
for k in numClusters:
    k_means = cluster.KMeans(n_clusters=k)
    k_means.fit(ds)
    SSE.append(k_means.inertia_)

plt.plot(numClusters, SSE)
plt.xlabel('Number of Clusters')
plt.ylabel('SSE')

In [ ]:
#K-means Clustering

Clus_ds = pd.DataFrame(ds)
Clus_ds

In [ ]:

#data = Clus_ds.drop('Summary',axis=1) <____ Not sure if we have to remove Summary or not!
k_means = cluster.KMeans(n_clusters=2, max_iter=50, random_state=1)
k_means.fit(data) 
labels = k_means.labels_
pd.DataFrame(labels, index=Clus_ds.Fatalities, columns=['Cluster ID'])

In [ ]:
centroids = k_means.cluster_centers_
pd.DataFrame(centroids,columns=data.columns)

In [ ]:
#TestData